In [1]:
import numpy as np
import torch

import sys
if "/home/yskim/projects/sparse-bfp" not in sys.path:
    sys.path.append("/home/yskim/projects/sparse-bfp")
from util.bfp.bfp_wrapper import BfpWrapper, BfpConfig
from util.custom_transpose import custom_transpose

In [2]:
def get_np(path):
    with open(path, "rb") as f:
        return np.load(f, allow_pickle=False)

In [11]:
data_root = "/home/yskim/projects/sparse-bfp/bfp-iter-log/resnet18-b32-fp-cuda"
fp_root = f"{data_root}/fp"
bfp_root = f"{data_root}/bfp-m8"

conv2d_num = 20
linear_num = 1
iterations = 20

phase = "fwd"

def get_bfp_outputs(bfp_path, layer_name, layer_id, phase):
    outputs = get_np(f"{bfp_path}/{layer_name}-{layer_id}-{phase}-bfp.npy")
    return outputs

def get_fp_outputs(fp_path, layer_name, layer_id, phase):
    outputs = get_np(f"{fp_path}/{layer_name}-{layer_id}-{phase}-fp.npy")
    return outputs

for i in range(iterations):
    print(f"[{phase}] {i}th iter")
    fp_path = f"{fp_root}/iter-{i}"
    bfp_path = f"{bfp_root}/iter-{i}"

    bfp_info = torch.load(f"{bfp_path}/iter_info.pth")
    # print(bfp_info)
    fp_info = torch.load(f"{fp_path}/iter_info.pth")
    # print(fp_info)

    # print(f"[train loss] fp: {fp_info['train_loss']}, bfp: {bfp_info['train_loss']}")

    for conv2d_id in range(1 if phase == "grad-a" else 0, conv2d_num):

        bfp_outputs = torch.tensor(get_bfp_outputs(bfp_path, "conv2d", conv2d_id, phase))
        bfp_outputs = custom_transpose(bfp_outputs, [0, 2, 3, 1]) # BHWC
        B, H, W, C = bfp_outputs.shape[0], bfp_outputs.shape[1], bfp_outputs.shape[2], bfp_outputs.shape[3]
        bfp_outputs = bfp_outputs.reshape(B * H * W, C)

        # print(bfp_outputs.shape)
        
        bfp_outputs_wrapper = BfpWrapper(bfp_outputs, should_sort=False)
        bfp_outputs_wrapper.run_convert_fp_to_bfp()


        bfp_cols = bfp_outputs.shape[1]
        bfp_nonzeros = np.count_nonzero(bfp_outputs.numpy(), axis=1)
        bfp_zeros = bfp_cols - bfp_nonzeros
        bfp_sparsity = bfp_zeros / bfp_cols * 100.0

        BfpConfig.bfp_M_Bit = 2
        M = bfp_outputs_wrapper.np_bfp_M
        M_cols = M.shape[1]
        M_nonzeros = np.count_nonzero(M, axis=1)
        M_zeros = M_cols - M_nonzeros
        M_sparsity = M_zeros / M_cols * 100.0

        print(f"[sparsity] output: {np.mean(bfp_sparsity):.2f}%, bfp converted: {np.mean(M_sparsity):.2f}%")


        # fp_outputs = get_fp_outputs(fp_path, "conv2d", conv2d_id, phase)

        # print(f"conv2d {conv2d_id:>3}th: [{bfp_lhs_M_shape[0]:>5}, {bfp_lhs_M_shape[1]:>5}] x [{bfp_rhs_M_shape[0]:>5} x {bfp_rhs_M_shape[1]:>5}], "
        #     #   f"[fp sparsity] lhs: {fp_lhs_sparsity:.2f}%, rhs: {fp_rhs_sparsity:.2f}%, [bfp sparsity] lhs {bfp_lhs_sparsity:.2f}%, rhs: {bfp_rhs_sparsity:.2f}%")
        #       f"[lhs sparsity] fp: {fp_lhs_sparsity:.2f}%, bfp: {bfp_lhs_sparsity:.2f}%, [rhs sparsity] fp: {fp_rhs_sparsity:.2f}%, bfp: {bfp_rhs_sparsity:.2f}%")


    # for linear_id in range(linear_num):
    #     lhs_E = get_np(f"{bfp_path}/linear-{linear_id}-{phase}-lhs-E-bfp.npy")
    #     lhs_ori_E = get_np(f"{bfp_path}/linear-{linear_id}-{phase}-lhs-ori-E-bfp.npy")
    #     lhs_M = get_np(f"{bfp_path}/linear-{linear_id}-{phase}-lhs-M-bfp.npy")

    #     rhs_E = get_np(f"{bfp_path}/linear-{linear_id}-{phase}-rhs-E-bfp.npy")
    #     rhs_ori_E = get_np(f"{bfp_path}/linear-{linear_id}-{phase}-rhs-ori-E-bfp.npy")
    #     rhs_M = get_np(f"{bfp_path}/linear-{linear_id}-{phase}-rhs-M-bfp.npy")

    #     print(f"linear {linear_id:>3}th: [{lhs_M.shape[0]:>5}, {lhs_M.shape[1]:>5}] x [{rhs_M.shape[0]:>5} x {rhs_M.shape[1]:>5}], "
    #           f"")

[fwd] 0th iter
[sparsity] output: 0.00%, bfp converted: 14.90%
[sparsity] output: 0.00%, bfp converted: 50.58%
[sparsity] output: 0.00%, bfp converted: 53.59%
[sparsity] output: 0.00%, bfp converted: 50.81%
[sparsity] output: 0.00%, bfp converted: 52.08%
[sparsity] output: 0.00%, bfp converted: 51.20%
[sparsity] output: 0.00%, bfp converted: 51.75%
[sparsity] output: 0.00%, bfp converted: 51.02%
[sparsity] output: 0.00%, bfp converted: 51.52%
[sparsity] output: 0.00%, bfp converted: 51.79%
[sparsity] output: 0.00%, bfp converted: 51.53%
[sparsity] output: 0.00%, bfp converted: 50.44%
[sparsity] output: 0.00%, bfp converted: 53.19%
[sparsity] output: 0.00%, bfp converted: 52.69%
[sparsity] output: 0.00%, bfp converted: 49.56%
[sparsity] output: 0.00%, bfp converted: 52.05%
[sparsity] output: 0.00%, bfp converted: 51.69%
[sparsity] output: 0.00%, bfp converted: 51.61%
[sparsity] output: 0.00%, bfp converted: 50.62%
[sparsity] output: 0.00%, bfp converted: 50.57%
[fwd] 1th iter
[sparsity]

In [ ]:
# lhs_E = get_np(f"{bfp_path}/{layer_name}-{layer_id}-{phase}-lhs-E-bfp.npy")
#     lhs_ori_E = get_np(f"{bfp_path}/{layer_name}-{layer_id}-{phase}-lhs-ori-E-bfp.npy")
#     lhs_E_offsets = lhs_E - lhs_ori_E
#     lhs_E_zero_indices = np.where(lhs_E_offsets >= 8)


#     lhs_M = get_np(f"{bfp_path}/{layer_name}-{layer_id}-{phase}-lhs-M-bfp.npy")

#     lhs_M_nonzeros = np.count_nonzero(lhs_M, axis=1)
#     lhs_cols = lhs_M.shape[1]
#     lhs_M_zeros = lhs_cols - lhs_M_nonzeros
#     lhs_sparsity = lhs_M_zeros / lhs_cols * 100.0

#     rhs_E = get_np(f"{bfp_path}/{layer_name}-{layer_id}-{phase}-rhs-E-bfp.npy")
#     rhs_ori_E = get_np(f"{bfp_path}/{layer_name}-{layer_id}-{phase}-rhs-ori-E-bfp.npy")
#     rhs_E_offsets = rhs_E - rhs_ori_E
#     rhs_E_zero_indices = np.where(rhs_E_offsets >= 8)